## Data Centric XAI part - 1
### CHAPTER 03 - *Data Centric Approaches*

From **Applied Machine Learning Explainability Techniques** by [**Aditya Bhattacharya**](https://www.linkedin.com/in/aditya-bhattacharya-b59155b6/), published by **Packt**

### Objective

In this notebook, we will try to implement some of the concepts related to Surrogate Explainers as part of the Result visualization based explainability methods discussed in Chapter 2 - Model Explainability Methods.

### Installing the modules

Install the following libraries in Google Colab or your local environment, if not already installed.

In [ ]:
!pip install --upgrade pandas numpy matplotlib seaborn scikit-learn xgboost skope-rules

### Loading the modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings("ignore")
np.random.seed(5)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree
import xgboost as xgb
import sklearn
import six
import sys
sys.modules['sklearn.externals.six'] = six
from skrules import SkopeRules
from dtreeviz.trees import dtreeviz
import graphviz

### About the data

**Adult Data Set - UCI Machine Learning Repository**

This dataset is also known as the *Census Income* dataset which is used to predict whether the income exceeds $50k/year based on census data. It is a multivariate dataset used for classification based problems containing 14 different features. More details about this data can be found at - [https://archive.ics.uci.edu/ml/datasets/adult](https://archive.ics.uci.edu/ml/datasets/adult)

### Loading the data

In [2]:
data  = pd.read_csv('Datasets/adult_census.csv', header=None)
data.columns = ['Age','Workclass','fnlwgt','Education','Education-Num','Marital Status','Occupation','Relationship','Race','Sex','Capital Gain','Capital Loss','Hours per week','Country','AnnualIncome'].copy()
data.head()

,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,AnnualIncome
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
data.shape

(32561, 15)

### Data Preprocessing

### Training a model

### Final Thoughts

The methods explored in this notebook, are all model agnostic approaches for explaining black-box models. These methods are very simple to execute and quite human friendly depending on the Surrogate Explainer methods used. Also, the result visualizations doesn't need any complicated level of understanding, so very effective for business stakeholders. But the major drawback is that these methods doesn't include the information learnt from the actual model. The actual model might learn important information and might unravel certain critical features which might not be picked up the Surrogate models. Overall, this is one of the most common methods for explaining black-box models, mainly because of its simplicity. Try out this method for regression based problems.

### Reference

1. UCI Machine Learning Repository -https://archive.ics.uci.edu/ml/datasets/adult
3. Some of the utility functions and code are taken from the GitHub Repository of the author - Aditya Bhattacharya https://github.com/adib0073